# Library

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras import Input
from keras.losses import binary_crossentropy
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from keras.callbacks import ModelCheckpoint

2023-06-28 15:22:28.797171: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-28 15:22:28.923326: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-28 15:22:28.951873: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-28 15:22:29.402445: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
xTrain=np.load("../Data/new_splitData/trainData.npy")
yTrain=np.load("../Data/new_splitData/trainTarget.npy")
xTest=np.load('../Data/new_splitData/testData.npy')
yTest=np.load("../Data/new_splitData/testTarget.npy")
xTrain_r=np.reshape(xTrain,(xTrain.shape[0],xTrain.shape[1]*2))
yTrain=np.reshape(yTrain,(yTrain.shape[0]))
xTest_r=np.reshape(xTest,(xTest.shape[0],xTest.shape[1]*2))
yTest=np.reshape(yTest,(yTest.shape[0]))
print("X shape: {}\nY shape: {}".format(xTrain_r.shape,yTrain.shape))

X shape: (320000, 500)
Y shape: (320000,)


# PCA

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(xTrain_r)
X_test = sc.transform(xTest_r)
pca = PCA(n_components = 100)
X_train = pca.fit_transform(X_train)
print (pca.explained_variance_)
print (pca.explained_variance_ratio_)
print (pca.explained_variance_ratio_.cumsum())
X_test = pca.transform(X_test)
print("X shape: {}".format(X_train.shape))

# NN

In [3]:
def arch():
    model=Sequential()
    model.add(Input(shape=(500)))
    model.add(Dense(512,activation='relu'))
    # model.add(Dropout(0.8))
    model.add(Dense(256,activation='relu'))
    # model.add(Dropout(0.8))
    model.add(Dense(128,activation='relu'))
    # model.add(Dropout(0.8))
    model.add(Dense(64,activation='relu'))
    # model.add(Dropout(0.8))
    model.add(Dense(32,activation='relu'))
    # model.add(Dropout(0.8))
    model.add(Dense(16,activation='relu'))
    # model.add(Dropout(0.8))
    model.add(Dense(4,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    return model
# arch().summary()


In [4]:
model=arch()
model.summary
opt=SGD( learning_rate=0.01, momentum=0.9)
model.compile(loss=binary_crossentropy, optimizer='adam', metrics=['accuracy'])
model_path="../Model/NN/{epoch:02d}-{val_accuracy:.4f}.h5"
checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

2023-06-28 15:22:36.570686: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-28 15:22:37.246756: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 45950 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:19:00.0, compute capability: 8.6


In [5]:
H=model.fit(xTrain_r,yTrain,
          validation_data=(xTest_r, yTest),
          epochs=500,batch_size=1024,
          callbacks=callbacks_list,
          verbose=1)

Epoch 1/500
 26/313 [=>............................] - ETA: 1s - loss: 0.6933 - accuracy: 0.5017

2023-06-28 15:22:43.467414: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


311/313 [============================>.] - ETA: 0s - loss: 0.6932 - accuracy: 0.4997
Epoch 1: val_accuracy improved from -inf to 0.49872, saving model to ../Model/NN/01-0.4987.h5
313/313 [==============================] - 4s 9ms/step - loss: 0.6932 - accuracy: 0.4996 - val_loss: 0.6932 - val_accuracy: 0.4987
Epoch 2/500
312/313 [============================>.] - ETA: 0s - loss: 0.6932 - accuracy: 0.5004
Epoch 2: val_accuracy did not improve from 0.49872
313/313 [==============================] - 2s 7ms/step - loss: 0.6932 - accuracy: 0.5004 - val_loss: 0.6932 - val_accuracy: 0.4987
Epoch 3/500
313/313 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5003
Epoch 3: val_accuracy improved from 0.49872 to 0.50128, saving model to ../Model/NN/03-0.5013.h5
313/313 [==============================] - 2s 8ms/step - loss: 0.6931 - accuracy: 0.5003 - val_loss: 0.6931 - val_accuracy: 0.5013
Epoch 4/500
310/313 [============================>.] - ETA: 0s - loss: 0.6932 - accura

KeyboardInterrupt: 

In [ ]:
N = 150
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_accuracy")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="test_accuracy")
plt.title("Qubit State Classification")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend(loc="lower left")